<a href="https://colab.research.google.com/github/Dharshika795/TNDSC_generative-ai/blob/main/smart%20agriculture%20system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dissolved-inorganic-nitrogen-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1648542%2F2706154%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240330%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240330T093541Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1402c445ce0c2877ac13ea9e602802dd3f0305d5baaff012e25d34b94548aab4b37b67643063bb9cb6b39327dff4c279f13a1cef34290da4701348c5a749deda3fcd76d8a4e5ab433599cf2023cf22ca69543788b92b6b4318d4dd86c98b6990166854bf041f6ff0908544bdd15b0c1ff6cd05770432b741d35c0412a87a71dc7ce2a78a8b0e4502df9b21aa4bff560f06cc3b306fbb8c1cab09cc5e882b7441ada856f0661c810f41da18276c3bd6740d3b9f2ab0b4171aea70eb7f4b632df45ac3865931ef183d996ef8bebe1ec10011826d869194c26bf0288e74d6e7a0cbf95df4fb6828ae746e90b7220593088106abeed156431713dcdcd3ee961babc6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**IMPORT THE NECESSARYLIBRARIES**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

**IMPORT THE DATA**

In [ ]:
dataset = pd.read_csv('../input/dissolved-inorganic-nitrogen-prediction/River Sites Nutrient Levels.csv')

Check the data

In [ ]:
dataset.head()

There appears to be an unnamed column in the dataset. This column must be dropped.

Found a null entry while scanning through the data.

In [ ]:
dataset['Water_electrical_conductivity'] = dataset['Water_electrical_conductivity'].fillna(dataset['Water_electrical_conductivity'].mean())

In [ ]:
plt.figure(figsize = (12,8))
plt.title('DIN at respective regions')
sns.stripplot(x = dataset['Region'],
              y = dataset['Dissolved_inorganic_nitrogen'])
plt.xlabel('Site on Swartkops River Estuary')
plt.ylabel('Dissolved inorganic nitrogen (µg/l)')
plt.show()

In [ ]:
plt.figure(figsize = (12,8))
plt.title('DIP at respective regions')
sns.stripplot(x = dataset['Region'],
              y = dataset['Dissolved_inorganic_phosphorus'])
plt.xlabel('Site on Swartkops River Estuary')
plt.ylabel('Dissolved inorganic phosphorus (µg/l)')
plt.show()

In [ ]:
plt.figure(figsize = (8,8))
plt.title('Relationship between physical properties and DIP')
g = sns.scatterplot(x = dataset['Water_electrical_conductivity'],
                y = dataset['Water_pH'],
               hue = dataset['Dissolved_inorganic_phosphorus'])
g.set(xlim=(0,2000),ylim=(0,10),xticks=[0,500, 1000, 1500], yticks=[0,2, 4, 6, 8, 10])
plt.ylabel('Water pH')
plt.xlabel('Electrical conductivity of water (S m-1)')
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
plt.title('Relationship between physical properties and DIN')
h = sns.scatterplot(x = dataset['Water_electrical_conductivity'],
                y = dataset['Water_pH'],
               hue = dataset['Dissolved_inorganic_nitrogen'])
h.set(xlim=(0,2000),ylim=(0,10),xticks=[0,500, 1000, 1500], yticks=[0,2, 4, 6, 8, 10])
plt.ylabel('Water pH')
plt.xlabel('Electrical conductivity of water (S m-1)')
plt.show()

**CREATE MATRIX OF FEATURES AND INDEPENDENT VARIABLE VECTOR**

In [ ]:
# Create a matrix of features (x; independant variables) and a dependant variable vector (y)
X = dataset.iloc[:, :-2].values
y = dataset.iloc[:, -2].values

In [ ]:
print(X)

In [ ]:
print(y)

**ENCODING THE CATEGORICAL DATA**

The first coloumn of the independent variables contains categorical data.

In [ ]:
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(drop = 'first'), [0])], remainder= 'passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

In [ ]:
type(X)

**SPLITTING THE DATA**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

**TRAINING THE RFR**

In [ ]:
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)

**PREDICTING THE RESULTS**

In [ ]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1),
                      y_test.reshape(len(y_test),1)),1))

**EVALUATING THE ACCURACY**

In [ ]:
r2_score(y_test, y_pred)

The large outliers in the dataset have made for this really low prediction accuracy.